# Dengue Analysis for Municipalities

## 0. Imports

### 0.1. Libraries

In [ ]:
# Data manipulation libraries
import pandas                as pd
import numpy                 as np

# Visualization libraries
import seaborn               as sns
import matplotlib            as mpl
import matplotlib.pyplot     as plt

from matplotlib              import cm
from matplotlib.colors       import Normalize

from mpl_toolkits.axes_grid1 import make_axes_locatable

# import plotly.express       as px
# import plotly.graph_objects as go

# Geo libraries
import geopandas             as gpd

# Other
import warnings
import sys
import os

### 0.2. Settings

In [ ]:
# Ignoring warnings
warnings.filterwarnings('ignore')

# Pandas Settings
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

# Directory Settings
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
src_path     = os.path.join(project_root, 'src')
sys.path.append(src_path)

# Visualization Settings
%matplotlib inline

# Set a style
mpl.style.use('ggplot')  

# Customize rcParams
mpl.rcParams.update({
    'axes.titlesize': 24,
    'axes.labelsize': 16,
    'axes.facecolor': 'white',
    'axes.linewidth': 1,
    'axes.grid': True,
    'xtick.labelsize': 14,
    'xtick.color': 'black',
    'ytick.labelsize': 14,
    'ytick.color': 'black',
    'legend.fontsize': 12,
    'lines.linewidth': 2,
    'lines.markersize': 6,
    'grid.color': 'lightgray',
    'grid.linestyle': '-',
    'grid.linewidth': 0.5,
    'font.size': 12,
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial'],
    'figure.titlesize': 24,
    'figure.figsize': (18, 9),
    'figure.dpi': 150
})

### 0.3. Functions

In [ ]:
def set_plot(fig, title, xlabel, ylabel):
    """
    Summary: This function sets the plot configuration for the initial plots. 
    Args:
        fig: the figure object.
        title: the title of the plot.
        xlabel: the label for the x axis.
        ylabel: the label for the y axis.
    Returns: figure with the image configuration.
    """
    # Customizing
    fig.set_title(title)
    fig.set_xlabel(xlabel, color='black')
    fig.set_ylabel(ylabel, color='black')
    fig.tick_params(left=False, bottom=False)

    return fig

### 0.4. Classes

### 0.5. Datasets

In [ ]:
# Casos
df_casos = pd.read_csv('../data/interim/mun_dengue_casos_amazonia_legal.csv')

# Óbitos
df_obitos = pd.read_csv('../data/interim/mun_dengue_obitos_amazonia_legal.csv')

In [ ]:
# População
df_pop = pd.read_csv('../data/interim/mun_populacao.csv')

In [ ]:
# GeoDataFrame MU Brasil
gdf = gpd.read_file('../data/interim/amazonia_legal.geojson')

In [ ]:
# Amazônia Legal Brasileira
alb = pd.read_csv('../data/interim/mun_amazonia_legal.csv')

## 1. Data Preparation

In [ ]:
# Organizando dados de população 

# População
df_pop = pd.read_csv('../data/interim/mun_populacao.csv')

# Lista de municípios da ALB
lista_mun_alb = alb.cod_ibge.to_list()

# Filtrando os dados
df_pop = df_pop[df_pop['cod_ibge'].isin(lista_mun_alb)]

# Ajustando os anos
df_pop['2023'] = df_pop['2022']

# Dropping 
df_pop.drop(columns=['nome_municipio', 'sigla_UF'], inplace=True)

# Melt
df_pop = pd.melt(df_pop, id_vars=['cod_ibge', 'id_mun'], var_name='ano', value_name='pop')

# Changing types
# df_pop = df_pop.astype('int64')

## 2. Data Exploration

## 3. Data Statistics